# Cluster differential expressed genes

In [1]:
library_load <- suppressMessages(
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Single R
        library(SingleCellExperiment),
        library(SingleR), 
        
        # Data 
        library(dplyr), 
        library(openxlsx), 
        
        # Plotting 
        library(ggplot2), 
        library(patchwork)
        
    )
)

In [2]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [3]:
# Source files
source("plotting_global.R")
source("bin/SeuratDEG.R")

In [4]:
options(warn=-1)

# Parameter settings

In [5]:
so_file <- "data/object/qc/so_qc_treatment_reg.rds"

# Import Seurat 

In [6]:
so <- readRDS(so_file)

# Cluster annotation 

In [7]:
ref <- readRDS("data/haemosphere/se_haemosphere.rds")

In [8]:
source("bin/SingleRQC.R")
so <- lapply(so, SingleRSeurat, ref=ref, cluster="SCT_snn_res.0.8")

Running SingleR in cluster mode

Running SingleR in cluster mode



In [9]:
set_factor_level <- function(so) {
    
    so$main_labels_cluster <- factor(so$main_labels_cluster, factor(names(color$main_labels_haemosphere)))
    so$fine_labels_cluster <- factor(so$fine_labels_cluster, factor(names(color$fine_labels_haemosphere)))
    
    return(so)
    
}

so <- lapply(so, set_factor_level)

# Compute cluster differential expressed genes 

In [ ]:
cluster_deg <- lapply(so, FindAllMarkers)

In [24]:
saveRDS(cluster_deg, "result/cluster_deg.rds")

In [55]:
so_nacl <- so[["NaCl"]]
cluster_deg_nacl <- cluster_deg[["NaCl"]]

so_cpg <- so[["NaCl"]]
cluster_deg_cpg <- cluster_deg[["NaCl"]]

In [56]:
cluster_deg_nacl <- dplyr::left_join(cluster_deg_nacl, unique(so_nacl@meta.data[, c("SCT_snn_res.0.8", "main_labels_cluster", "fine_labels_cluster")]), by=c("cluster"="SCT_snn_res.0.8"))

cluster_deg_cpg <- dplyr::left_join(cluster_deg_cpg, unique(so_cpg@meta.data[, c("SCT_snn_res.0.8", "main_labels_cluster", "fine_labels_cluster")]), by=c("cluster"="SCT_snn_res.0.8"))

In [57]:
cluster_deg_nacl <- split(cluster_deg_nacl, f=cluster_deg_nacl$cluster)
cluster_deg_nacl <- cluster_deg_nacl[as.character(0:(length(cluster_deg_nacl)-1))]
names(cluster_deg_nacl) <- paste0("cluster_", names(cluster_deg_nacl))

cluster_deg_cpg <- split(cluster_deg_cpg, f=cluster_deg_cpg$cluster)
cluster_deg_cpg <- cluster_deg_cpg[as.character(0:(length(cluster_deg_cpg)-1))]
names(cluster_deg_cpg) <- paste0("cluster_", names(cluster_deg_cpg))

In [61]:
write.xlsx(cluster_deg_nacl, file="result/cluster_deg_nacl.xlsx", overwrite = TRUE)

write.xlsx(cluster_deg_cpg, file="result/cluster_deg_cpg.xlsx", overwrite = TRUE)

# Compute dim reduction plots 

In [ ]:
dim_plot <- function(so, reduction="umap", cluster="SCT_snn_res.0.8") {
    
    dplot_1 <- DimPlot(so, reduction=reduction, group.by=cluster, label=TRUE) & 
        theme(aspect.ratio=1, legend.position="none")
    
    dplot_2 <- DimPlot(so, reduction=reduction, group.by="tissue", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$tissue, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
    dplot_3 <- DimPlot(so, reduction=reduction, group.by="cc_phase_class", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$cc_phase_class, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
        # Combine plots 
    dplot <- dplot_1 + dplot_2 + dplot_3 + 
        plot_layout(ncol=3) 
    
}

In [ ]:
dim_plot_label <- function(so, reduction="umap", cluster="SCT_snn_res.0.8") {
    
    dplot_1 <- DimPlot(so, reduction=reduction, group.by=cluster, label=TRUE) & 
        theme(aspect.ratio=1, legend.position="none")
    
    dplot_2 <- DimPlot(so, reduction=reduction, group.by="main_labels", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$main_labels, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
    dplot_3 <- DimPlot(so, reduction=reduction, group.by="fine_labels", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$fine_labels, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
        # Combine plots 
    dplot <- dplot_1 + dplot_2 + dplot_3 + 
        plot_layout(ncol=3)
    
}

In [ ]:
dim_plot_label_cluster <- function(so, reduction="umap", cluster="SCT_snn_res.0.8") {
    
    dplot_1 <- DimPlot(so, reduction=reduction, group.by=cluster, label=TRUE) & 
        theme(aspect.ratio=1, legend.position="none")
    
    dplot_2 <- DimPlot(so, reduction=reduction, group.by="main_labels_cluster", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$main_labels, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
    dplot_3 <- DimPlot(so, reduction=reduction, group.by="fine_labels_cluster", label=FALSE) & 
        theme(aspect.ratio=1, legend.position="bottom") & 
        scale_color_manual(values=color$fine_labels, na.value="dark gray") & 
        guides(color=guide_legend(ncol=3, override.aes=list(size=2)))
    
        # Combine plots 
    dplot <- dplot_1 + dplot_2 + dplot_3 + 
        plot_layout(ncol=3) 
    
}

# Dim plot NaCl

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_nacl <- dim_plot(so[["NaCl"]])
dim_plot_nacl
ggsave(dim_plot_nacl, filename="result/plot/cluster_deg/dim_plot_nacl.png", width=15, height=10)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_label_nacl <- dim_plot_label(so[["NaCl"]])
dim_plot_label_nacl
ggsave(dim_plot_label_nacl, filename="result/plot/cluster_deg/dim_plot_label_nacl.png", width=15, height=10)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_label_cluster_nacl <- dim_plot_label_cluster(so[["NaCl"]])
dim_plot_label_cluster_nacl
ggsave(dim_plot_label_cluster_nacl, filename="result/plot/cluster_deg/dim_plot_label_cluster_nacl.png", width=15, height=10)

# Dim plot CpG 

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_cpg <- dim_plot(so[["CpG"]])
dim_plot_cpg
ggsave(dim_plot_cpg, filename="result/plot/cluster_deg/dim_plot_cpg.png", width=15, height=10)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_label_cpg <- dim_plot_label(so[["CpG"]])
dim_plot_label_cpg
ggsave(dim_plot_label_cpg, filename="result/plot/cluster_deg/dim_plot_label_cpg.png", width=15, height=10)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
dim_plot_label_cluster_cpg <- dim_plot_label_cluster(so[["CpG"]])
dim_plot_label_cluster_cpg
ggsave(dim_plot_label_cluster_cpg, filename="result/plot/cluster_deg/dim_plot_label_cluster_cpg.png", width=15, height=10)